In [1]:
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer

In [2]:
afq_dataset = AFQDataset(
    fn_nodes="../data/raw/combined_tract_profiles.csv",
    fn_subjects="../data/raw/participants_updated_id.csv",
    dwi_metrics=["dki_fa", "dki_md"],
    target_cols=["age"],
    index_col="subject_id",
)

In [3]:
afq_dataset.drop_target_na()

In [4]:
print(len(afq_dataset.subjects))
print(afq_dataset.X.shape)
print(afq_dataset.y.shape)

1867
(1867, 4800)
(1867,)


In [5]:
# Here we impute missing bundles.
# We impute using the entire dataset, which permits data leakage between the train and test set.
# THIS IS BAD AND SHOULDN'T BE DONE IN PRODUCTION
# But we do it here to move straight to model training
# When we are more comfortabel with the models, we should come back
# here and train the imputer only on the training set.
imputer = SimpleImputer(strategy="median")
afq_dataset.X = imputer.fit_transform(afq_dataset.X)

In [7]:
# Separate into training, test, and validation sets
dataset_size = len(afq_dataset.subjects)
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = int(0.15 * dataset_size)

full_dataset = afq_dataset.as_tensorflow_dataset()
full_dataset = full_dataset.shuffle(buffer_size=dataset_size, seed=0)

train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(val_size)
test_dataset = test_dataset.take(test_size)

In [8]:
batch_size = 128
train_dataset = train_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [12]:
def hist(model):
    result_cycles = np.array([])
    for i in range(10):
        model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
        model.fit(train_dataset, epochs=100, validation_data=val_dataset)
        result_cycles = np.append(result_cycles, model.evaluate(val_dataset)[0])
    return result_cycles

In [44]:
# a = list([1,2])
# c = np.array([])
# np.append(c, a)

In [13]:
# First let's check out the ``cnn_lenet`` from the tf_models module
model_cnn_lenet = cnn_lenet(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)

pooling layers: 4
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 48)]         0         
                                                                 
 conv1d_4 (Conv1D)           (None, 100, 6)            870       
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 50, 6)            0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 50, 16)            304       
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 25, 16)           0         
 1D)                                                             
                                                                 
 conv1d_6 (Conv1D)           (None, 25, 2

In [14]:
arr = hist(model_cnn_lenet)

Epoch 1/100
11/11 [==============================] - 1s 39ms/step - loss: 115.3572 - mean_squared_error: 115.3572 - val_loss: 80.0709 - val_mean_squared_error: 80.0709
Epoch 2/100
11/11 [==============================] - 0s 25ms/step - loss: 42.8404 - mean_squared_error: 42.8404 - val_loss: 20.6040 - val_mean_squared_error: 20.6040
Epoch 3/100
11/11 [==============================] - 0s 26ms/step - loss: 27.4524 - mean_squared_error: 27.4524 - val_loss: 24.0568 - val_mean_squared_error: 24.0568
Epoch 4/100
11/11 [==============================] - 0s 27ms/step - loss: 23.9747 - mean_squared_error: 23.9747 - val_loss: 12.3826 - val_mean_squared_error: 12.3826
Epoch 5/100
11/11 [==============================] - 0s 27ms/step - loss: 22.2696 - mean_squared_error: 22.2696 - val_loss: 12.8015 - val_mean_squared_error: 12.8015
Epoch 6/100
11/11 [==============================] - 0s 26ms/step - loss: 22.0530 - mean_squared_error: 22.0530 - val_loss: 15.3884 - val_mean_squared_error: 15.3884
Ep

Epoch 50/100
11/11 [==============================] - 0s 26ms/step - loss: 15.8383 - mean_squared_error: 15.8383 - val_loss: 8.6350 - val_mean_squared_error: 8.6350
Epoch 51/100
11/11 [==============================] - 0s 26ms/step - loss: 15.9833 - mean_squared_error: 15.9833 - val_loss: 10.4742 - val_mean_squared_error: 10.4742
Epoch 52/100
11/11 [==============================] - 0s 27ms/step - loss: 16.0553 - mean_squared_error: 16.0553 - val_loss: 9.2075 - val_mean_squared_error: 9.2075
Epoch 53/100
11/11 [==============================] - 0s 26ms/step - loss: 14.8085 - mean_squared_error: 14.8085 - val_loss: 8.5658 - val_mean_squared_error: 8.5658
Epoch 54/100
11/11 [==============================] - 0s 26ms/step - loss: 15.3492 - mean_squared_error: 15.3492 - val_loss: 11.6880 - val_mean_squared_error: 11.6880
Epoch 55/100
11/11 [==============================] - 0s 27ms/step - loss: 15.9208 - mean_squared_error: 15.9208 - val_loss: 9.5348 - val_mean_squared_error: 9.5348
Epoch 

11/11 [==============================] - 0s 26ms/step - loss: 12.6655 - mean_squared_error: 12.6655 - val_loss: 8.4065 - val_mean_squared_error: 8.4065
Epoch 100/100
3/3 [==============================] - 0s 4ms/step - loss: 8.2635 - mean_squared_error: 8.2635
Epoch 1/100
11/11 [==============================] - 1s 40ms/step - loss: 13.3305 - mean_squared_error: 13.3305 - val_loss: 7.9841 - val_mean_squared_error: 7.9841
Epoch 2/100
11/11 [==============================] - 0s 27ms/step - loss: 11.9228 - mean_squared_error: 11.9228 - val_loss: 8.9148 - val_mean_squared_error: 8.9148
Epoch 3/100
11/11 [==============================] - 0s 26ms/step - loss: 11.9531 - mean_squared_error: 11.9531 - val_loss: 7.7524 - val_mean_squared_error: 7.7524
Epoch 4/100
11/11 [==============================] - 0s 26ms/step - loss: 12.1678 - mean_squared_error: 12.1678 - val_loss: 9.6276 - val_mean_squared_error: 9.6276
Epoch 5/100
11/11 [==============================] - 0s 26ms/step - loss: 11.6740 -

11/11 [==============================] - 0s 26ms/step - loss: 10.2324 - mean_squared_error: 10.2324 - val_loss: 7.4566 - val_mean_squared_error: 7.4566
Epoch 49/100
11/11 [==============================] - 0s 26ms/step - loss: 10.9362 - mean_squared_error: 10.9362 - val_loss: 10.3262 - val_mean_squared_error: 10.3262
Epoch 50/100
11/11 [==============================] - 0s 26ms/step - loss: 10.2935 - mean_squared_error: 10.2935 - val_loss: 7.7746 - val_mean_squared_error: 7.7746
Epoch 51/100
11/11 [==============================] - 0s 26ms/step - loss: 10.6448 - mean_squared_error: 10.6448 - val_loss: 7.3968 - val_mean_squared_error: 7.3968
Epoch 52/100
11/11 [==============================] - 0s 26ms/step - loss: 11.2449 - mean_squared_error: 11.2449 - val_loss: 7.5364 - val_mean_squared_error: 7.5364
Epoch 53/100
11/11 [==============================] - 0s 26ms/step - loss: 10.9688 - mean_squared_error: 10.9688 - val_loss: 7.1313 - val_mean_squared_error: 7.1313
Epoch 54/100
11/11 [=

11/11 [==============================] - 0s 29ms/step - loss: 9.0135 - mean_squared_error: 9.0135 - val_loss: 7.2338 - val_mean_squared_error: 7.2338
Epoch 98/100
11/11 [==============================] - 0s 26ms/step - loss: 9.6328 - mean_squared_error: 9.6328 - val_loss: 6.6609 - val_mean_squared_error: 6.6609
Epoch 99/100
11/11 [==============================] - 0s 26ms/step - loss: 9.2493 - mean_squared_error: 9.2493 - val_loss: 6.1532 - val_mean_squared_error: 6.1532
Epoch 100/100
3/3 [==============================] - 0s 6ms/step - loss: 6.1109 - mean_squared_error: 6.1109
Epoch 1/100
11/11 [==============================] - 1s 40ms/step - loss: 10.0068 - mean_squared_error: 10.0068 - val_loss: 6.2216 - val_mean_squared_error: 6.2216
Epoch 2/100
11/11 [==============================] - 0s 26ms/step - loss: 9.8524 - mean_squared_error: 9.8524 - val_loss: 6.5899 - val_mean_squared_error: 6.5899
Epoch 3/100
11/11 [==============================] - 0s 26ms/step - loss: 9.2804 - mean_s

...

11/11 [==============================] - 0s 25ms/step - loss: 4.0472 - mean_squared_error: 4.0472 - val_loss: 2.2774 - val_mean_squared_error: 2.2774
Epoch 98/100
11/11 [==============================] - 0s 25ms/step - loss: 4.0710 - mean_squared_error: 4.0710 - val_loss: 2.9184 - val_mean_squared_error: 2.9184
Epoch 99/100
11/11 [==============================] - 0s 25ms/step - loss: 4.3121 - mean_squared_error: 4.3121 - val_loss: 2.6240 - val_mean_squared_error: 2.6240
Epoch 100/100
3/3 [==============================] - 0s 4ms/step - loss: 2.4589 - mean_squared_error: 2.4589
Epoch 1/100
11/11 [==============================] - 1s 37ms/step - loss: 4.8445 - mean_squared_error: 4.8445 - val_loss: 2.6689 - val_mean_squared_error: 2.6689
Epoch 2/100
11/11 [==============================] - 0s 25ms/step - loss: 4.5619 - mean_squared_error: 4.5619 - val_loss: 3.1316 - val_mean_squared_error: 3.1316
Epoch 3/100
11/11 [==============================] - 0s 26ms/step - loss: 3.7445 - mean_squ

Epoch 47/100
11/11 [==============================] - 0s 32ms/step - loss: 3.6903 - mean_squared_error: 3.6903 - val_loss: 2.0534 - val_mean_squared_error: 2.0534
Epoch 48/100
11/11 [==============================] - 0s 29ms/step - loss: 3.7514 - mean_squared_error: 3.7514 - val_loss: 2.1477 - val_mean_squared_error: 2.1477
Epoch 49/100
11/11 [==============================] - 0s 30ms/step - loss: 3.8643 - mean_squared_error: 3.8643 - val_loss: 1.7397 - val_mean_squared_error: 1.7397
Epoch 50/100
11/11 [==============================] - 0s 42ms/step - loss: 4.0251 - mean_squared_error: 4.0251 - val_loss: 2.4424 - val_mean_squared_error: 2.4424
Epoch 51/100
11/11 [==============================] - 0s 26ms/step - loss: 4.0723 - mean_squared_error: 4.0723 - val_loss: 2.4970 - val_mean_squared_error: 2.4970
Epoch 52/100
11/11 [==============================] - 0s 26ms/step - loss: 4.4199 - mean_squared_error: 4.4199 - val_loss: 2.1933 - val_mean_squared_error: 2.1933
Epoch 53/100
11/11 [==

11/11 [==============================] - 0s 27ms/step - loss: 3.7123 - mean_squared_error: 3.7123 - val_loss: 2.1076 - val_mean_squared_error: 2.1076
Epoch 98/100
11/11 [==============================] - 0s 27ms/step - loss: 3.6910 - mean_squared_error: 3.6910 - val_loss: 1.9764 - val_mean_squared_error: 1.9764
Epoch 99/100
11/11 [==============================] - 0s 26ms/step - loss: 3.6596 - mean_squared_error: 3.6596 - val_loss: 2.1117 - val_mean_squared_error: 2.1117
Epoch 100/100
3/3 [==============================] - 0s 6ms/step - loss: 2.2788 - mean_squared_error: 2.2788
Epoch 1/100
11/11 [==============================] - 1s 39ms/step - loss: 5.4792 - mean_squared_error: 5.4792 - val_loss: 3.9295 - val_mean_squared_error: 3.9295
Epoch 2/100
11/11 [==============================] - 0s 26ms/step - loss: 4.7111 - mean_squared_error: 4.7111 - val_loss: 2.1097 - val_mean_squared_error: 2.1097
Epoch 3/100
11/11 [==============================] - 0s 26ms/step - loss: 3.7602 - mean_squ

Epoch 47/100
11/11 [==============================] - 0s 26ms/step - loss: 3.8460 - mean_squared_error: 3.8460 - val_loss: 2.0000 - val_mean_squared_error: 2.0000
Epoch 48/100
11/11 [==============================] - 0s 26ms/step - loss: 3.9539 - mean_squared_error: 3.9539 - val_loss: 1.8405 - val_mean_squared_error: 1.8405
Epoch 49/100
11/11 [==============================] - 0s 27ms/step - loss: 3.4708 - mean_squared_error: 3.4708 - val_loss: 1.7541 - val_mean_squared_error: 1.7541
Epoch 50/100
11/11 [==============================] - 0s 26ms/step - loss: 3.6929 - mean_squared_error: 3.6929 - val_loss: 1.9117 - val_mean_squared_error: 1.9117
Epoch 51/100
11/11 [==============================] - 0s 26ms/step - loss: 3.7738 - mean_squared_error: 3.7738 - val_loss: 2.2762 - val_mean_squared_error: 2.2762
Epoch 52/100
11/11 [==============================] - 0s 27ms/step - loss: 3.5877 - mean_squared_error: 3.5877 - val_loss: 1.6337 - val_mean_squared_error: 1.6337
Epoch 53/100
11/11 [==

11/11 [==============================] - 0s 26ms/step - loss: 3.5052 - mean_squared_error: 3.5052 - val_loss: 2.4469 - val_mean_squared_error: 2.4469
Epoch 98/100
11/11 [==============================] - 0s 26ms/step - loss: 3.5666 - mean_squared_error: 3.5666 - val_loss: 2.1733 - val_mean_squared_error: 2.1733
Epoch 99/100
11/11 [==============================] - 0s 26ms/step - loss: 3.9701 - mean_squared_error: 3.9701 - val_loss: 1.8213 - val_mean_squared_error: 1.8213
Epoch 100/100
3/3 [==============================] - 0s 6ms/step - loss: 1.7155 - mean_squared_error: 1.7155


In [15]:
# plt.hist(arr)
# # plt.title('100 times')
# plt.xlabel('mean-squared-error')
# plt.ylabel('frequency')
arr

array([8.26348305, 6.1108923 , 5.52248096, 6.63443851, 3.46258473,
       3.34933448, 2.40332532, 2.45892978, 2.27878022, 1.71545708])